## BERT（Bidirectional Encoder Representations from Transformers）


<img src="../data/img/bert.png"  style="zoom:30%"/>


BERT的设计理念主要基于以下几点：

- 双向性（Bidirectional）: 与传统的单向语言模型不同，BERT能同时考虑到词语的前后文。
- 通用性（Generality）: 通过预训练和微调的方式，BERT能适用于多种自然语言处理任务。
- 深度（Depth）: BERT通常具有多层（通常为12层或更多），这使得模型能够捕捉复杂的语义和语法信息。

<img src="../data/img/bert_arc.png"  style="zoom:50%"/>


### 嵌入层（Embedding Layer）
BERT使用了Token Embeddings, Segment Embeddings和Position Embeddings三种嵌入方式，将输入的单词和附加信息编码为固定维度的向量。

- Token Embedding：对input中的所有词汇（包括正常词汇和特殊词汇）都做Embedding，如随机初始化
- Segment Embedding：由于处理的是两个句子，故需要对两个句子进行区分。第一个句子的token全部用0来表示，第二个句子的token全部用1来表示，让模型得以判断上下句的起止位置
- Position Embedding：与Transformer的输入部分（Positional Encoding）有很大不同。Transformer中用的是正余弦函数固定好的，而BERT中是使用随机初始化，模型自己学习出每个位置的Embedding
  
<img src="../data/img/bert_embedding.png"  style="zoom:50%"/>

Input：[CLS]上一句话[SEP]下一句话[SEP]。也就是在句子开头加一个[CLS]，在两句话之间和句末加[SEP]

- 正常词汇：##ing表示是BERT分词器分词之后的结果，如图中playing本来是一个完整的单词，但由于在词库中出现频率不高，被切分成play和ing这两个词根。##就是代表被切分的这个词本身应该与前面的词合在一起，组成一个完整的词
- 特殊词汇：BERT预训练中有一个任务是NSP（Next Sentence Prediction，判断两个句子之间的关系）二分类任务
    -  [CLS]：在句子最前面加该符号，训练时将[CLS]的输出向量接一个二分类器，去做一个二分类任务
    - [SEP]：告诉模型该符号前是一个句子，该符号后是另一个句子
  
> 在预训练完之后，[CLS]输出向量并不能代表整个句子的语义信息\
>[CLS]向量用在NSP任务中，是一个二分类任务，和编码整个句子的语义信息任务相去甚远\
>故用[CLS]输出向量去无监督地做文本相似度任务时，效果会非常差

### 如何对BERT做预训练：MLM+NSP
Google 在预训练BERT时让它同时进行两个任务：

- Masked Language Model，即MLM（完形填空）
- Next Sentence Prediction，即NSP（判断第二个句子在原始本文中是否跟第一个句子相接）

<img src="../data/img/bert_MLM_NSP.png"  style="zoom:50%"/>

<img src="../data/img/bert_MLM_task.png"  style="zoom:50%"/>

BERT在预训练时使用的是大量无标注的语料，故在预训练任务设计时要考虑无监督（因为无标签）。对于无监督任务来说，有两种目标函数：

- AR（Auto-Regressive）：自回归模型；只能考虑单侧的信息，典型的就是GPT
- AE（Auto-Encoding）：自编码模型；从损坏的输入数据中预测、重建原始数据，可以使用上下文的信息。

BERT使用的目标函数就是AE（自编码模型）

<img src="../data/img/bert_NSP_task.png"  style="zoom:50%"/>

为什么用[CLS]的输出，而不是first sentence等的输出来做分类？
A：Self-Attention在计算词和词之间的权重时，自己和自己的分数是比较大的，不希望分类结果含有单独某个词的意思。而[CLS]本身无意义，就可以很纯粹地含有两个句子里的信息

重点：理解样本的构造模式

NSP任务的样本如下:

- 从训练语料库中取出两个连续的段落作为正样本（两个段落来自同一文档/主题，且顺序没有颠倒）
- 从不同的文档中随机创建一对段落作为负样本（不同的主题随机抽一个段落，连接在一起）

缺点：主题预测和连贯性预测合并为一个单项任务

- 主题预测：判断两个文本是不是来自同一个文档
- 连贯性预测：判断两个段落是不是顺序关系

## Fine-Tune

<img src="../data/img/bert_finetune.png"  style="zoom:50%"/>

BERT模型的成功很大程度上归功于其两阶段的训练策略：预训练（Pre-training）和微调（Fine-tuning）。
- (a) 文本匹配就是把两个句子拼接起来，去判断它是否相似，也是用[CLS]输出去判断0（不相似）或1（相似）
- (b) 单个句子的文本分类，使用第一个[CLS]的输出去做一个微调，二分类或多分类
- (c) QA任务
- (d) 序列标注，就是把所有的token输出做一个softmax，去看它属于实体中的哪一个

### 预训练（Pre-training）

任务
- 掩码语言模型（Masked Language Model, MLM）: 在这个任务中，输入句子的某个比例的词会被随机地替换成特殊的[MASK]标记，模型需要预测这些被掩码的词。
- 下一个句子预测（Next Sentence Prediction, NSP）: 模型需要预测给定的两个句子是否是连续的。

技术点:
- 动态掩码: 在每个训练周期（epoch）中，模型看到的每一个句子的掩码都是随机的，这样可以增加模型的鲁棒性。
- 分词器: BERT使用了WordPiece分词器，能有效处理未登录词（OOV）。

注意点:

- 数据规模需要非常大，以充分训练庞大的模型参数。
- 训练过程通常需要大量的计算资源，例如高性能的GPU或TPU。

### 微调（Fine-tuning）
在预训练模型好之后，接下来就是微调阶段。微调通常在具有标签的小规模数据集上进行，以使模型更好地适应特定的任务。

技术点:
- 学习率调整: 由于模型已经在大量数据上进行了预训练，因此微调阶段的学习率通常会设置得相对较低。
- 任务特定头: 根据任务的不同，通常会在BERT模型的顶部添加不同的网络层（例如，用于分类任务的全连接层、用于序列标记的CRF层等）。

注意点:

- 避免过拟合：由于微调数据集通常比较小，因此需要仔细选择合适的正则化策略，如Dropout或权重衰减（weight decay）。
- 通过这两个阶段的训练，BERT不仅能够捕捉到丰富的语义和语法信息，还能针对特定任务进行优化，从而在各种NLP任务中都表现得非常出色。


<img src="../data/img/bert_finetune_case1.png"  style="zoom:50%"/>

<img src="../data/img/bert_finetune_case2.png"  style="zoom:50%"/>

<img src="../data/img/bert_finetune_case3.png"  style="zoom:50%"/>

<img src="../data/img/bert_finetune_case4.png"  style="zoom:50%"/>

<img src="../data/img/bert_finetune_case4_1.png"  style="zoom:50%"/>


### 如何提升BERT在下游任务中的表现
先Domain Transfer（领域），再Task Transfer（任务），最后Fine-tune（微调），性能是最好的

- 在大量通用语料上训练一个Language Model（Pre-train，预训练模型）—— 一般不做，直接用中文谷歌BERT
- 在相同领域的文本上继续训练Language Model（Domain transfer，领域迁移/领域自适应） —— 在大量微博文本上继续训练这个BERT
- 在任务相关的小数据上继续训练Language Model（Task transfer，任务迁移）—— 在微博情感文本上（第2步中有的文本不属于情感分析的范畴）
- 在任务相关数据上做具体任务（Fine-tune，微调）

如何在相同领域数据中进行further pre-training？第2步中可以使用的一些trick：

- 动态mask：BERT在训练时使用的是固定的mask，即把文本mask后存在本地，每次训练时都是使用这同一个文件，每次训练的都是使用同样的mask标志的文本。动态mask即每个epoch训练之前对数据mask，每个epoch时mask的单词很大概率是不同的，而不是一直使用同一个文件
- n-gram mask：比如ERNIE和SpanBERT都是类似于做了实体词的mask。若自己训练时没有特别准确的实体词，可以不做实体词的mask，而是做n-gram mask
- 参数设置：
    - batch_size：16/32/64/128 —— 影响不太大，看机器性能
    - learning rate（Adam）：5e-5/3e-5/2e-5，尽可能小一点，避免灾难性遗忘
    - number of epochs：3/4 —— 微调时epoch一般不会太大
    - weighted decay修改后的Adam：使用warmup，搭配线性衰减
- 预训练时做一些数据增强（EDA）、自蒸馏、外部知识融入（知识图谱/实体词信息等）